# Importing and loading the dataset

In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
import random
from tqdm import tqdm
import cv2 as cv
import PIL
from PIL import Image
!pip install plotly
import plotly.express as px
from IPython import display

import tensorflow as tf
from tensorflow import keras

import torch
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unzip data

Archive:  data.zip
replace imgs/imgs/0.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
base_dir = '/content/'
os.listdir(base_dir)

In [ ]:
data_dir = '/content/txn_history-2021-10-07.jsonl'
image_dir = "/content/imgs/imgs"
image_root = "/content/imgs"

In [ ]:
df = pd.read_json(base_dir + 'txn_history-2021-10-07.jsonl', lines=True)
df.head()

In [ ]:
no_plots = 10*10

images = glob.glob("/content/imgs/imgs/*.png")

plt.rcParams['figure.figsize'] = (30, 30)
plt.subplots_adjust(wspace=0, hspace=0)

print("Sample 100 CryptoPunks")
for idx,image in enumerate(images[:no_plots]):
    sample_img = cv.imread(image)
    plt.subplot(10, 10, idx+1)
    plt.axis('off')
    plt.imshow(cv.cvtColor(sample_img,cv.COLOR_BGR2RGB)) 
plt.show()

In [ ]:
punks = [img for img in glob.glob("/content/imgs/imgs/*.png")]

for punk in punks[0:3]:
    img = plt.imread(punk)
    plt.imshow(img)
    plt.show()

In [ ]:
df.info()

In [ ]:
df = df[["txn_type", "date", "eth", "punk_id", "type", "accessories"]]
df.head()

In [ ]:
df['txn_type'].unique()

In [ ]:
df['accessories'].explode().unique()

In [ ]:
len(df['accessories'].explode().unique())

In [ ]:
df = df.explode("type")
df.head()

# DataLoader and Preprocess

In [ ]:
def tensor_imshow(img, dnorm=True):
    img = img.to('cpu')
    npimg = img.detach().numpy()
    if dnorm:
        npimg = npimg*0.5+0.5
    plt.figure(figsize=(3, 3))
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
def get_dataloader(batch_size,           
                   image_size,           
                   data_dir=image_dir,   
                   num_workers=3):       
    
    stats = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) 
  
    transform = transforms.Compose([transforms.Resize((image_size, image_size)),  
                                    transforms.ToTensor(),                        
                                    transforms.Normalize(*stats)])                
    dataset = datasets.ImageFolder(root=data_dir,
                                   transform=transform)
    data_loader = torch.utils.data.DataLoader(dataset,
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=num_workers,
                                              pin_memory=True)
    
    return data_loader

In [ ]:
batch_size, image_size = 5, 24

train_loader = get_dataloader(batch_size,
                              image_size,
                              image_root)

dataiter = iter(train_loader) 
img,_ = next(dataiter)
sample_img = img[-1]

tensor_imshow(sample_img)

# Build Generator

In [ ]:
class Generator(nn.Module):      
    def __init__(self, 
                 z_dim=100,      
                 im_chan=3,      
                 hidden_dim=64): 
        
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.im_chan = im_chan
        self.hidden_dim = hidden_dim
        
        self.generator_cnn = nn.Sequential(self.make_gen_block(z_dim, hidden_dim*8, stride=1, padding=0),   
                                           self.make_gen_block(hidden_dim*8, hidden_dim*4),                           
                                           self.make_gen_block(hidden_dim*4, hidden_dim*2),                           
                                           self.make_gen_block(hidden_dim*2, hidden_dim),                             
                                           self.make_gen_block(hidden_dim, im_chan, final_layer=True))
    
    def make_gen_block(self, 
                       im_chan,    
                       op_chan,     
                       kernel_size=4, 
                       stride=2, 
                       padding=1, 
                       final_layer=False): 
        
        layers = []
        layers.append(nn.ConvTranspose2d(im_chan,     
                                         op_chan, 
                                         kernel_size, 
                                         stride, 
                                         padding, 
                                         bias=False))
        
        if not final_layer:
            layers.append(nn.BatchNorm2d(op_chan))
            layers.append(nn.LeakyReLU(0.2))
        else:
            layers.append(nn.Tanh())
        
        return nn.Sequential(*layers)
    
    def forward(self,noise):
        x = noise.view(-1,self.z_dim,1,1)
        return self.generator_cnn(x)

    def get_noise(n_samples, 
                  z_dim, 
                  device='cpu'):
        return torch.randn(n_samples, 
                           z_dim, 
                           device=device)

In [ ]:
noise = Generator.get_noise(n_samples=5,
                            z_dim=100)

g = Generator(z_dim=100,
              im_chan=3,
              hidden_dim=64)

In [ ]:
print(g)

# Build Discrimnator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, 
                 im_chan=3,      
                 conv_dim=64,     
                 image_size=64):  
        
        super(Discriminator, self).__init__()
        self.image_size = image_size
        self.conv_dim = conv_dim
        
        self.disc_cnn = nn.Sequential(self.make_disc_block(im_chan, conv_dim),
                                      self.make_disc_block(conv_dim, conv_dim*2),
                                      self.make_disc_block(conv_dim*2, conv_dim*4),
                                      self.make_disc_block(conv_dim*4, conv_dim*8),
                                      self.make_disc_block(conv_dim*8, 1, padding=0, final_layer=True)) 
        
        
    def make_disc_block(self,
                        im_chan,
                        op_chan,
                        kernel_size=4,
                        stride=2,
                        padding=1,
                        final_layer=False):
        layers = []
        layers.append(nn.Conv2d(im_chan,
                                op_chan,
                                kernel_size,
                                stride,
                                padding,
                                bias=False))
        
        if not final_layer:
            layers.append(nn.BatchNorm2d(op_chan))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
        
        return nn.Sequential(*layers)
    def forward(self,image):
        pred = self.disc_cnn(image)
        pred = pred.view(image.size(0),-1)
        return pred
    
    def _get_final_feature_dimention(self):
        final_width_height = (self.image_size //  2**len(self.disc_cnn))**2
        final_depth = self.conv_dim * 2**(len(self.disc_cnn)-1)
        return final_depth*final_width_height

In [ ]:
d = Discriminator(im_chan=3,
                  conv_dim=64,
                  image_size=64)

In [ ]:
print(d)

#Define Training Parameters

In [ ]:
def weights_init_normal(m):
    
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02) 
        
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)

In [ ]:
def real_loss(D_out,device='cpu'):
    
    criterion = nn.BCEWithLogitsLoss()
    

    batch_size = D_out.size(0)
 
    labels = torch.ones(batch_size, device=device)*0.9 
    
    loss = criterion(D_out.squeeze(), labels)
    return loss

In [ ]:
def fake_loss(D_out, device='cpu'):
    
 
    criterion = nn.BCEWithLogitsLoss()
    
  
    batch_size = D_out.size(0)
    
    labels = torch.zeros(batch_size,
                         device=device) 
    
    loss = criterion(D_out.squeeze(), labels)
    return loss

# Training 

In [ ]:
def print_tensor_images(images_tensor):
    
    '''
    Function for visualizing images: Given a tensor of images, prints the images.
    '''
        
    plt.rcParams['figure.figsize'] = (15, 15)
    plt.subplots_adjust(wspace=0, hspace=0)
    
    images_tensor = images_tensor.to('cpu')
    npimgs = images_tensor.detach().numpy()
    
    no_plots = len(images_tensor)

    for idx,image in enumerate(npimgs):
        plt.subplot(1, 8, idx+1)
        plt.axis('off')
        image = image * 0.5 + 0.5
        plt.imshow(np.transpose(image, (1, 2, 0)))
        
    plt.show()

In [ ]:
def train(D, G, 
          n_epochs,
          dataloader,
          d_optimizer,
          g_optimizer,
          z_dim,
          print_every=50,
          device='cpu'):
    
    sample_size=8
    fixed_z = Generator.get_noise(n_samples=sample_size,
                                  z_dim=z_dim,
                                  device=device)
    
    for epoch in range(1,n_epochs+1):
        for batch_i,(real_images,_) in enumerate(dataloader):
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            
            d_optimizer.zero_grad()

            d_real_op = D(real_images)
            d_real_loss = real_loss(d_real_op,
                                    device=device)

            noise = Generator.get_noise(n_samples=batch_size,
                                        z_dim=z_dim,
                                        device=device)
            fake_images = G(noise)

            d_fake_op = D(fake_images) 
            d_fake_loss = fake_loss(d_fake_op,
                                    device=device)

            d_loss = d_real_loss + d_fake_loss
            
       
            d_loss.backward()
            d_optimizer.step()

            g_optimizer.zero_grad()
            noise = Generator.get_noise(n_samples=batch_size,
                                        z_dim=z_dim,
                                        device=device)

            g_out = G(noise)
            d_out = D(g_out)
            
            g_loss = real_loss(d_out, 
                               device=device) 
            g_loss.backward()
            g_optimizer.step()
        
        print('Epoch [{:5d}/{:5d}] | d_loss: {:6.4f} | g_loss: {:6.4f}'.format(epoch, 
                                                                               n_epochs, 
                                                                               d_loss.item(),  
                                                                               g_loss.item())) 
        if (epoch % print_every == 0):
            G.eval()
            sample_image = G(fixed_z)
            print_tensor_images(sample_image)
            G.train()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device is ", device)

z_dim = 100       
beta_1 = 0.5    
beta_2 = 0.999 
lr = 0.0002      
n_epochs = 100
batch_size = 128
image_size = 64

In [ ]:
generator = Generator(z_dim, 
                      im_chan=3, 
                      hidden_dim=64).to(device)

discriminator = Discriminator(im_chan=3, 
                              conv_dim=64, 
                              image_size=image_size).to(device)

g_optimizer = optim.Adam(generator.parameters(), 
                         lr=lr, 
                         betas=(beta_1, beta_2))

d_optimizer = optim.Adam(discriminator.parameters(), 
                         lr=lr, 
                         betas=(beta_1, beta_2))

dataloader = get_dataloader(batch_size, 
                            image_size, 
                            image_root)

In [ ]:
%time
n_epochs = 100
train(discriminator,
      generator,
      n_epochs,
      dataloader,
      d_optimizer,
      g_optimizer,
      z_dim,
      print_every=10,
      device=device)

In [ ]:
plt.figure(figsize = (15, 8))
plt.plot(noise)
plt.title("Noise")
plt.show()

#Create new cryptopunks

In [ ]:
generator.to(device)
generator.eval()      
sample_size=8

for i in range(8):    
    
    fixed_z = Generator.get_noise(n_samples=sample_size, 
                                  z_dim=z_dim, 
                                  device=device)    
    
    sample_image = generator(fixed_z)

    print_tensor_images(sample_image)